In [ ]:
import numpy as np
import gym
import gym_kheperaposition
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Input, Concatenate, BatchNormalization, Add
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
import math

# Se prepara el entorno
ENV_NAME = 'KheperaPositionObstacle-v0'
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
env.Training = True

nb_actions = env.action_space.shape[0]
nb_observation = env.observation_space.shape[0]
max_Lin_Vel, max_Ang_Vel = env.action_space.high

tanh_init = tf.random_uniform_initializer(minval=-0.02, maxval=0.02)
sigm_init = tf.random_uniform_initializer(minval=0.6, maxval=1)

# Se crean los modelos de Actor y Critic
inputs = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
out = Flatten()(inputs)
out = Dense(512, activation="relu")(out)
out = Dense(512, activation="relu")(out)
out = Dense(512, activation="relu")(out)
Lin_Vel = Dense(1, activation="sigmoid", kernel_initializer=sigm_init)(out)
Ang_Vel = layers.Dense(1, activation="tanh", kernel_initializer=tanh_init)(out)
outputs = Concatenate()([Lin_Vel,Ang_Vel])

Actor = Model(inputs, outputs)
print(Actor.summary())

observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
observation_inputs = Flatten()(observation_input)
observation_inputs = Dense(512, activation="relu")(observation_inputs)

action_input = Input(shape=nb_actions, name='action_input')

concat = Concatenate()([observation_inputs, action_input])

observation_out = Dense(512, activation="relu")(concat)
observation_out = Dense(512, activation="relu")(observation_out)
state_outputs = Dense(1, activation="linear")(observation_out)

Critic = Model([observation_input, action_input], state_outputs)
print(Critic.summary())

memory = SequentialMemory(limit=800000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=.15, mu=0., sigma=.3)

agent = DDPGAgent(nb_actions=nb_actions, actor=Actor, critic=Critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=10000, nb_steps_warmup_actor=10000,
                  random_process=random_process, gamma=.99, target_model_update=1e-3)

agent.compile(Adam(lr=.00005, clipnorm=1.), metrics=['mae'])

# Se cragan los pesos del agente
agent.load_weights('Weights/ddpg.h5')

In [ ]:
# Se realiza un Test del agente
env.MaxSteps = 1200
env.Training = False  # Para poder elegir la posicion del robot y el TP
agent.test(env, nb_episodes=1, visualize=False)